# Я-Метрика. Создание сегмента с ClientId

In [1]:
import pandas as pd
import datetime
# import os
import requests
import json

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as snsу

from tapi_yandex_metrika import YandexMetrikaLogsapi

# import sys

# Вывод всех операций в Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Счетчик и Токен от Яндекс_метрики
counterId = 11111111111
token_YM = '*****************************************'

In [3]:
headers = {
    "Host": "api-metrika.yandex.net",
    "Authorization": "OAuth " + token_YM,
    "Content-Type": "application/x-yametrika+json",
    "Content-Length": "123"
}

# Вывести список всех сегментов

In [81]:
# https://yandex.ru/dev/metrika/doc/api2/management/segments/getsegmentsforcounter.html

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/apisegment/segments"

req = requests.get(URL, headers=headers)
req
req.json()

<Response [200]>

{'segments': [{'segment_id': 1006408060,
   'counter_id': 65071099,
   'name': 'test___segment_Moskow_01',
   'expression': "ym:s:regionCityName=='Москва'",
   'is_retargeting': 0,
   'status': 'active',
   'segment_source': 'api'},
  {'segment_id': 1006408065,
   'counter_id': 65071099,
   'name': 'test___segment_100_clientId_02',
   'expression': 'ym:s:clientID=.(1635183259603994884,1635670848276114856,1636062370638530867,163676853957067267,1638961661982897343,1639383140478454322,16411185171016557127,1641375793879766591,1641383544291001492,1642001033657107255,164227067976281529,16424018641032314086,1642404528289704290,1643226839516253466,1643400424449465062,164420916790781202,1644928245945155011,16455837591047876072,164580251334529016,1646024402621012978,1646500946173387263,1646660462802604385,1647100180560696153,1647190015769537901,16472084331054026875,1647546426133562927,1647677037522505173,1647687945796367095,1647805568952256920,164807322975691257,1648114936170886667,1648135077457

# Удалить сегмент

In [75]:
# https://yandex.ru/dev/metrika/doc/api2/management/segments/deletesegment.html

segmentId = 1006407935

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/apisegment/segment/{segmentId}"

req = requests.delete(URL, headers=headers)
req
req.json()

<Response [404]>

{'errors': [{'error_type': 'not_found',
   'message': 'Object with id = 1006407935 not found'}],
 'code': 404,
 'message': 'Object with id = 1006407935 not found'}

# Создать сегмент (стандартный способ)

In [74]:
# https://yandex.ru/dev/metrika/doc/api2/management/segments/createsegment.html
# https://yandex.ru/dev/metrika/doc/api2/api_v1/segmentation.html

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/apisegment/segments"

body = {}
body['segment'] = {}
body['segment']['name'] = "test___segment_Moskow_01"
body['segment']['expression'] = "ym:s:regionCityName=='Москва'"
body['segment']['segment_source'] = "api"

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"segment":{"segment_id":1006408060,"counter_id":65071099,"name":"test___segment_Moskow_01","expression":"ym:s:regionCityName==\'Москва\'","is_retargeting":0,"status":"active","segment_source":"api"}}'

### Создаем условие из массива clientId

In [17]:
file_name = "csv/mobile_2.csv" 
df = pd.read_csv(file_name)
df.head()
df.shape

,clientid
0,1635183259603994884
1,1635670848276114856
2,1636062370638530867
3,163676853957067267
4,1638961661982897343


(1152, 1)

In [77]:
clientid_list = df['clientid'].to_list()

# len(clientid_list)
clientid_list_100 = list(map(str, clientid_list[:100]))
len(clientid_list_100)
str_many_clientId = ",".join(clientid_list_100)
str_many_clientId

100

'1635183259603994884,1635670848276114856,1636062370638530867,163676853957067267,1638961661982897343,1639383140478454322,16411185171016557127,1641375793879766591,1641383544291001492,1642001033657107255,164227067976281529,16424018641032314086,1642404528289704290,1643226839516253466,1643400424449465062,164420916790781202,1644928245945155011,16455837591047876072,164580251334529016,1646024402621012978,1646500946173387263,1646660462802604385,1647100180560696153,1647190015769537901,16472084331054026875,1647546426133562927,1647677037522505173,1647687945796367095,1647805568952256920,164807322975691257,1648114936170886667,1648135077457037442,1648138953517040253,1648209174615164115,1648221820319329413,1648232446654511881,1648295848145159721,16483566311041498106,1648461624258282963,1648617178804762302,1648630116298002036,16486819648618239,1648776658323456777,1648970908183712648,1648988450873033582,1649076939296858580,1649128623951938151,1649145988289711060,164917562029565144,1649190126473272939,16

# Создать сегмент (на базе короткого списка clientId - 100 элементов)

**Внимание!** Опытным путем выяснил, что лимит - 100 записей

In [78]:
# https://yandex.ru/dev/metrika/doc/api2/management/segments/createsegment.html
# https://yandex.ru/dev/metrika/doc/api2/api_v1/segmentation.html

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/apisegment/segments"

body = {}
body['segment'] = {}
body['segment']['name'] = "test___segment_100_clientId_02"
# Задать условие на один фильтр с массивом ClientID можно так: ym:s:clientID=.(11111, 2222, ...)
# Лимит: количество уникальных группировок и метрик — до 10, 
# количество отдельных фильтров — до 20, 
# длина строки в фильтре — до 10 000 символов
body['segment']['expression'] = f"ym:s:clientID=.({str_many_clientId})"
# "ym:s:clientID=='1635183259603990000'"
body['segment']['segment_source'] = "api"

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"segment":{"segment_id":1006408065,"counter_id":65071099,"name":"test___segment_100_clientId_02","expression":"ym:s:clientID=.(1635183259603994884,1635670848276114856,1636062370638530867,163676853957067267,1638961661982897343,1639383140478454322,16411185171016557127,1641375793879766591,1641383544291001492,1642001033657107255,164227067976281529,16424018641032314086,1642404528289704290,1643226839516253466,1643400424449465062,164420916790781202,1644928245945155011,16455837591047876072,164580251334529016,1646024402621012978,1646500946173387263,1646660462802604385,1647100180560696153,1647190015769537901,16472084331054026875,1647546426133562927,1647677037522505173,1647687945796367095,1647805568952256920,164807322975691257,1648114936170886667,1648135077457037442,1648138953517040253,1648209174615164115,1648221820319329413,1648232446654511881,1648295848145159721,16483566311041498106,1648461624258282963,1648617178804762302,1648630116298002036,16486819648618239,1648776658323456777,1648970908183

# ======== Передача параметров посетителей ========

# Список загрузок параметров посетителей

In [80]:
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/findall.html

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/user_params/uploadings"

req = requests.get(URL, headers=headers)
req
req.json()

<Response [200]>

{'uploadings': [{'id': 8012726,
   'create_time': '2022-07-20T05:07:37.000Z',
   'content_id_type': 'client_id',
   'action': 'update',
   'status': 'processed',
   'line_quantity': 1152,
   'linked_quantity': 25},
  {'id': 8012716,
   'create_time': '2022-07-20T04:53:02.000Z',
   'content_id_type': 'client_id',
   'action': 'update',
   'status': 'processed',
   'line_quantity': 1152,
   'linked_quantity': 0},
  {'id': 8003498,
   'create_time': '2022-07-17T17:41:28.000Z',
   'content_id_type': 'client_id',
   'action': 'update',
   'status': 'processed',
   'line_quantity': 3,
   'linked_quantity': 1},
  {'id': 8003430,
   'create_time': '2022-07-17T17:18:19.000Z',
   'content_id_type': 'client_id',
   'action': 'update',
   'status': 'processing_failed',
   'line_quantity': 5,
   'linked_quantity': 0}]}

# Выполняет загрузку параметров посетителей из CSV-файла

In [56]:
# Создадим файл-источник
# df = pd.read_csv("csv/1.csv")
# df['key'] = "test"
# df['value'] = 1
# df.to_csv("csv/mobile_2_plus.csv", index=False)

In [57]:
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/upload.html
file_name = "csv/mobile_2_plus.csv"

# action - тип действия, которое вы хотите выполнить.
# Допустимые значения:
# delete_keys — удаление параметров
# update — добавление параметров
URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/user_params/uploadings/upload?action=update"
headers = {"Authorization": f"OAuth {token_YM}"}
file = open(file_name,"r").read()
body = {'file': file} 

req = requests.post(URL, files=body, headers=headers)
req
req.text

<Response [200]>

'{"uploading":{"id":8012726,"action":"update","comment":"file","line_quantity":1152}}'

In [10]:
file

'clientid,key,value\n29864873,gender,F\n29864873,discount-card,1\n29864873,offline-purchase,5\n93657832,gender,М\n93657832,offline-purchase,3'

# Возвращает информацию об указанной загрузке параметров посетителей

In [52]:
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/findbyid.html
uploadingId = 8003430

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/user_params/uploading/{uploadingId}"

req = requests.get(URL, headers=headers)
req
req.json()

<Response [200]>

{'uploading': {'id': 8003430,
  'create_time': '2022-07-17T17:18:19.000Z',
  'content_id_type': 'client_id',
  'action': 'update',
  'status': 'processing_failed',
  'line_quantity': 5,
  'linked_quantity': 0}}

# Подтверждение загрузки параметров

In [58]:
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/confirm.html

# Идентификатор загрузки параметров посетителей, которую вы хотите подтвердить
uploadingId = 8012726

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/user_params/uploading/{uploadingId}/confirm"

body = {}
body['uploading'] = {}

# Тип идентификаторов посетителей. Возможные значения:
# user_id — идентификатор посетителя сайта, назначенный владельцем сайта.
# client_id — идентификатор посетителя сайта, назначенный Яндекс.Метрикой.
body['uploading']['content_id_type'] = 'client_id' 

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"uploading":{"id":8012726,"create_time":"2022-07-20T05:07:37.000Z","content_id_type":"client_id","action":"update","status":"is_processed","line_quantity":1152,"linked_quantity":0}}'

# Создаём сегмент (на базе загрузки параметров пользователя)

In [64]:
# https://yandex.ru/dev/metrika/doc/api2/management/segments/createsegment.html
# https://yandex.ru/dev/metrika/doc/api2/api_v1/segmentation.html

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/apisegment/segments"

body = {}
body['segment'] = {}
body['segment']['name'] = "test__segment_param_user_03"

# body['segment']['expression'] = "ym:s:regionCityName=='Тула'"
# Задать параметр посетителей со значением child=1 нужно вот так:
# ym:up:paramsLevel1=='value1' and ym:up:paramsLevel2=='value2', 
# где value1 - параметр первого уровня (child), value2 - параметр второго уровня ('1').
body['segment']['expression'] = "ym:up:paramsLevel1=='test' and ym:up:paramsLevel2=='1'"
body['segment']['segment_source'] = "api"

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.json()

<Response [200]>

{'segment': {'segment_id': 1006407935,
  'counter_id': 65071099,
  'name': 'test__segment_param_user_03',
  'expression': "ym:up:paramsLevel1=='test' and ym:up:paramsLevel2=='1'",
  'is_retargeting': 0,
  'status': 'active',
  'segment_source': 'api'}}

# Изменение загрузки параметров (удаление)

In [40]:
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/update.html
# https://yandex.ru/dev/metrika/doc/api2/management/userparams/class_userparamsuploading.html

uploadingId = 8003498

URL = f"https://api-metrika.yandex.net/management/v1/counter/{counterId}/user_params/uploading/{uploadingId}"

body = {}
body['uploading'] = {}
# id-загрузки
body['uploading']['id'] = 8003498
body['uploading']['content_id_type'] = 'client_id' 
# Действие, которое необходимо выполнить.
# Возможные значения
# delete_keys — удаление параметров.
# update — добавление параметров.
body['uploading']['action'] = 'delete_keys'

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.put(URL, body, headers=headers)
req
req.json()

<Response [200]>

{'uploading': {'id': 8003498,
  'create_time': '2022-07-17T17:41:28.000Z',
  'content_id_type': 'client_id',
  'action': 'update',
  'status': 'processed',
  'line_quantity': 3,
  'linked_quantity': 1}}